<a href="https://colab.research.google.com/github/SimranSingh98/Digantara_Assignment/blob/main/Digantara_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Parsing the TLE Data**

In [ ]:
pip install sgp4

In [2]:
def read_tle_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()

    tle_data = []
    for i in range(0, len(lines), 3):
        title_line = lines[i].strip()
        line1 = lines[i + 1].strip()
        line2 = lines[i + 2].strip()
        tle_data.append((title_line, line1, line2))

    return tle_data

# Given File
filename = '30000sats.txt'
tle_data = read_tle_file(filename)

# Displaying parsed TLE data
# for tle in tle_data:
#     print("Title:", tle[0])
#     print("Line 1:", tle[1])
#     print("Line 2:", tle[2])
#     print()


**1. Get Satellite location**

In [ ]:
import csv
from sgp4.api import Satrec, jday
from datetime import datetime, timedelta

def get_satellite_location(tle_data, output_file):
    # now = datetime.utcnow()
    # Open the CSV file within the context of the `with` block
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['time', 'Lx', 'Ly', 'Lz', 'Vx', 'Vy', 'Vz']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for title, line1, line2 in tle_data:
            satellite = Satrec.twoline2rv(line1, line2)

            # Set start_time close to the TLE epoch (30sats - 54th day, 30000sats - 238th day)
            start_time = datetime(2023, 8, 25, 0, 0, 0)
            end_time = start_time + timedelta(days=1)

            current_time = start_time
            while current_time <= end_time:
                jd, fr = jday(current_time.year, current_time.month, current_time.day,
                              current_time.hour, current_time.minute, current_time.second)

                error_code, r, v = satellite.sgp4(jd, fr)

                if error_code == 0:
                    # Write the result to the CSV file
                    writer.writerow({
                        'time': current_time.isoformat(),
                        'Lx': r[0],
                        'Ly': r[1],
                        'Lz': r[2],
                        'Vx': v[0],
                        'Vy': v[1],
                        'Vz': v[2]
                    })
                else:
                    print(f"Error in SGP4 propagation for {title}: {error_code}")

                current_time += timedelta(minutes=1)


output_file = 'satellite_positions_and_velocities.csv'
get_satellite_location(tle_data, output_file)

**2. Convert data to lat long alt format**

In [ ]:

import csv
import pyproj

import pyproj

def ecef2lla(i, pos_x, pos_y, pos_z):
    # Define the ECEF projection based on WGS84 ellipsoid
    ecef = pyproj.Proj(proj="geocent", ellps="WGS84", datum="WGS84")

    # Define the LLA (latitude, longitude, altitude) projection
    lla = pyproj.Proj(proj="latlong", ellps="WGS84", datum="WGS84")

    # Create a transformer object to convert ECEF to LLA
    transformer = pyproj.Transformer.from_proj(ecef, lla)

    # Transform ECEF (X, Y, Z) to LLA (longitude, latitude, altitude)
    lona, lata, alta = transformer.transform(pos_x[i], pos_y[i], pos_z[i], radians=False)

    # Return the transformed coordinates
    return lona, lata, alta


# Step 1: Read the CSV file and extract Lx, Ly, Lz values
input_file = 'satellite_positions_and_velocities.csv'
Lx, Ly, Lz = [], [], []

with open(input_file, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        Lx.append(float(row['Lx']))
        Ly.append(float(row['Ly']))
        Lz.append(float(row['Lz']))

# Step 2: Convert the extracted ECEF coordinates to LLA format
# start_time = time.time()

lla_results = [ecef2lla(i, Lx, Ly, Lz) for i in range(len(Lx))]

# end_time = time.time()
# print(f"filter_results execution time: {end_time - start_time:.2f} seconds")

# Step 3: Output the LLA results
output_file = 'satellite_latlongalt.csv'
with open(output_file, 'w', newline='') as csvfile:
    fieldnames = ['Longitude', 'Latitude', 'Altitude']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for lon, lat, alt in lla_results:
        writer.writerow({'Longitude': lon, 'Latitude': lat, 'Altitude': alt})

# Print the LLA results (optional)
# for lon, lat, alt in lla_results:
#     print(f"Longitude: {lon}, Latitude: {lat}, Altitude: {alt}")


**3. Find when it is going from over certain lat long region.**

In [ ]:
import csv

def is_within_bounds(lat, lon, lat_min, lat_max, lon_min, lon_max):
    """Check if a given lat/lon is within the specified bounds."""
    return lat_min <= lat <= lat_max and lon_min <= lon <= lon_max

def filter_results(input_file, output_file):
    # Ask user for the four coordinates
    # print("Enter the latitude and longitude for the four corners of the rectangle:")

    lat1 = 100.66673 #float(input("Latitude 1: "))
    lon1 = 20.58196 #float(input("Longitude 1: "))
    lat2 = 100.74973 #float(input("Latitude 2: "))
    lon2 = -12.64459 #float(input("Longitude 2: "))
    lat3 = -100.09096 #float(input("Latitude 3: "))
    lon3 = 50.71009 #float(input("Longitude 3: "))
    lat4 = -100.32309 #float(input("Latitude 4: "))
    lon4 = 100.79778 #float(input("Longitude 4: "))

    # Determine the bounding box
    lat_min = min(lat1, lat2, lat3, lat4)
    lat_max = max(lat1, lat2, lat3, lat4)
    lon_min = min(lon1, lon2, lon3, lon4)
    lon_max = max(lon1, lon2, lon3, lon4)

    print(f"Bounding box: Lat ({lat_min}, {lat_max}), Lon ({lon_min}, {lon_max})")

    # Open the CSV file containing the LLA data
    with open(input_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        filtered_results = []

        # Print some sample data
        print("Sample data:")
        for i, row in enumerate(reader):
            if i < 5:  # Print the first 5 rows as a sample
                print(row)
            lat = float(row['Latitude'])
            lon = float(row['Longitude'])

            # Filter based on the bounding box
            if is_within_bounds(lat, lon, lat_min, lat_max, lon_min, lon_max):
                filtered_results.append(row)

    # Write the filtered results to a new CSV file
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['Longitude', 'Latitude', 'Altitude']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in filtered_results:
            writer.writerow(row)

    # Optionally, print filtered results
    # if filtered_results:
    #     print(f"Filtered results ({len(filtered_results)} entries):")
    #     for row in filtered_results:
    #         print(f"Longitude: {row['Longitude']}, Latitude: {row['Latitude']}, Altitude: {row['Altitude']}")
    # else:
    #     print("No results found within the specified bounds.")

# Input and output files
input_file = 'satellite_latlongalt.csv'
output_file = 'filtered_satellite_positions.csv'

# Call the function to filter results based on user input
filter_results(input_file, output_file)


Enter the latitude and longitude for the four corners of the rectangle:
Bounding box: Lat (-100.32309, 100.74973), Lon (-12.64459, 100.79778)
Sample data:
{'Longitude': '28.55127073937771', 'Latitude': '-90.0', 'Altitude': '-6352498.518419408'}
{'Longitude': '32.62178825418735', 'Latitude': '-90.0', 'Altitude': '-6352291.224795465'}
{'Longitude': '36.97259264989291', 'Latitude': '-90.0', 'Altitude': '-6352104.268830082'}
{'Longitude': '41.621819950967634', 'Latitude': '-90.0', 'Altitude': '-6351938.497879223'}
{'Longitude': '46.58021900573466', 'Latitude': '-90.0', 'Altitude': '-6351794.6630765265'}


**4. Optimize code to reduce computation time**

In [ ]:
# Not using GPU (just multi-threading)

from multiprocessing import Pool
from datetime import datetime, timedelta
from sgp4.api import Satrec, jday
import csv
import time

# Function to get satellite position and velocity for a single satellite
def get_satellite_location_single(tle):
    title_line, tle1, tle2 = tle  # Unpacking three elements now
    satellite = Satrec.twoline2rv(tle1, tle2)
    results = []
    now = datetime(2023, 2, 24, 0, 0, 0)
    for minute in range(1440):  # 24 hours * 60 minutes
        time = now + timedelta(minutes=minute)
        jd, fr = jday(time.year, time.month, time.day, time.hour, time.minute, time.second)
        e, r, v = satellite.sgp4(jd, fr)
        results.append((time, r[0], r[1], r[2], v[0], v[1], v[2]))
    return results

# Function to handle parallel processing
def get_satellite_location_parallel(tle_data):
    with Pool() as pool:
        results = pool.map(get_satellite_location_single, tle_data)
    return [item for sublist in results for item in sublist]

# Function to write results to a CSV file
def write_results_to_csv(results, filename):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Time', 'Lx', 'Ly', 'Lz', 'Vx', 'Vy', 'Vz'])
        writer.writerows(results)

# Main execution block
if __name__ == "__main__":
    start_time = time.time()

    # Read TLE data
    filename = '30sats.txt'
    tle_data_large = read_tle_file(filename)

    # Calculate satellite positions in parallel
    satellite_positions_optimized = get_satellite_location_parallel(tle_data_large)

    # Write results to CSV
    write_results_to_csv(satellite_positions_optimized, 'satellite_positions_gpu.csv')

    # Calculate and print the execution time
    execution_time = time.time() - start_time
    print(f"Execution Time: {execution_time:.2f} seconds")


Execution Time: 0.82 seconds


In [ ]:
# Install CuPy with CUDA 12.x support
!pip install cupy-cuda12x


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 MB 9.0 MB/s eta 0:00:00


**1. Get Satellite location**

In [ ]:
import cupy as cp
import numpy as np
from datetime import datetime, timedelta
from sgp4.api import Satrec, jday
import time
import csv

def compute_julian_dates(start_time, minutes):
    """Compute Julian Dates for a range of minutes on CPU."""
    time_deltas = np.arange(minutes) * 60  # Compute the number of seconds for each minute
    times = [start_time + timedelta(seconds=int(delta)) for delta in time_deltas]
    jd, fr = zip(*[jday(t.year, t.month, t.day, t.hour, t.minute, t.second) for t in times])
    return np.array(jd), np.array(fr)

def get_satellite_location_gpu(tle_data):
    results = []

    now = datetime(2023, 2, 24, 0, 0, 0)
    minutes = 1440  # 24 hours * 60 minutes

    # Precompute Julian Dates on CPU
    jd_cpu, fr_cpu = compute_julian_dates(now, minutes)

    # Transfer Julian Dates to GPU
    jd_gpu = cp.array(jd_cpu, dtype=cp.float64)
    fr_gpu = cp.array(fr_cpu, dtype=cp.float64)

    for tle in tle_data:
        title_line, tle1, tle2 = tle
        satellite = Satrec.twoline2rv(tle1, tle2)

        # Pre-allocate space for the results
        positions = cp.zeros((minutes, 6), dtype=cp.float64)  # 3 for position (x, y, z) and 3 for velocity (vx, vy, vz)

        # Run SGP4 on the GPU for all times
        for i in range(minutes):
            e, r, v = satellite.sgp4(jd_gpu[i].item(), fr_gpu[i].item())
            positions[i, :3] = cp.array(r, dtype=cp.float64)   # Convert tuple to CuPy array
            positions[i, 3:] = cp.array(v, dtype=cp.float64)

        results.append(cp.asnumpy(positions))

    return results

# Main execution block
if __name__ == "__main__":
    start_time = time.time()  # Correctly initialize start_time with time.time()

    # Calculate satellite positions using GPU
    satellite_positions_gpu = get_satellite_location_gpu(tle_data)

    # Optionally: Write results to a CSV file
    with open('satellite_positions_gpu.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Lx', 'Ly', 'Lz', 'Vx', 'Vy', 'Vz'])
        for positions in satellite_positions_gpu:
            writer.writerows(positions)

    # Calculate and print the execution time
    execution_time = time.time() - start_time
    print(f"Execution Time: {execution_time:.2f} seconds")


**2. Convert data to lat long alt format**.

In [ ]:
import csv
import pyproj
import numpy as np
import cupy as cp
import time

def ecef2lla_cpu(pos_x, pos_y, pos_z):
    """Convert ECEF coordinates to LLA using the CPU."""
    # Define the ECEF projection based on WGS84 ellipsoid
    ecef = pyproj.Proj(proj="geocent", ellps="WGS84", datum="WGS84")

    # Define the LLA (latitude, longitude, altitude) projection
    lla = pyproj.Proj(proj="latlong", ellps="WGS84", datum="WGS84")

    # Create a transformer object to convert ECEF to LLA
    transformer = pyproj.Transformer.from_proj(ecef, lla)

    # Convert GPU arrays to NumPy arrays
    pos_x_np = cp.asnumpy(pos_x)
    pos_y_np = cp.asnumpy(pos_y)
    pos_z_np = cp.asnumpy(pos_z)

    # Transform ECEF (X, Y, Z) to LLA (longitude, latitude, altitude) on CPU
    lona, lata, alta = transformer.transform(pos_x_np, pos_y_np, pos_z_np, radians=False)

    return lona, lata, alta

# Step 1: Read the CSV file and extract Lx, Ly, Lz values
input_file = 'satellite_positions_gpu.csv'
Lx, Ly, Lz = [], [], []

with open(input_file, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        Lx.append(float(row['Lx']))
        Ly.append(float(row['Ly']))
        Lz.append(float(row['Lz']))

# Convert lists to CuPy arrays
Lx_cp = cp.array(Lx)
Ly_cp = cp.array(Ly)
Lz_cp = cp.array(Lz)

# Step 2: Convert the extracted ECEF coordinates to LLA format
start_time = time.time()

# Perform the conversion using CPU (after transferring data from GPU)
lona, lata, alta = ecef2lla_cpu(Lx_cp, Ly_cp, Lz_cp)

end_time = time.time()
print(f"Conversion execution time: {end_time - start_time:.2f} seconds")

# Step 3: Output the LLA results to a CSV file
output_file = 'satellite_latlongalt_gpu.csv'
with open(output_file, 'w', newline='') as csvfile:
    fieldnames = ['Longitude', 'Latitude', 'Altitude']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    for lon, lat, alt in zip(lona, lata, alta):
        writer.writerow({'Longitude': lon, 'Latitude': lat, 'Altitude': alt})

# Optionally: Print the LLA results
# for lon, lat, alt in zip(lona, lata, alta):
#     print(f"Longitude: {lon}, Latitude: {lat}, Altitude: {alt}")


Conversion execution time: 0.02 seconds


**3. Find when it is going from over certain lat long
region.**.

In [ ]:
import csv
import cupy as cp
import numpy as np
import time

# Define CUDA kernel for filtering
filter_kernel = cp.RawKernel(r'''
extern "C" __global__
void filter_data(const float* latitudes, const float* longitudes,
                 float lat_min, float lat_max, float lon_min, float lon_max,
                 int* mask, int num_elements) {
    int idx = threadIdx.x + blockIdx.x * blockDim.x;
    if (idx < num_elements) {
        float lat = latitudes[idx];
        float lon = longitudes[idx];
        if (lat >= lat_min && lat <= lat_max && lon >= lon_min && lon <= lon_max) {
            mask[idx] = 1;
        } else {
            mask[idx] = 0;
        }
    }
}
''', 'filter_data')

def load_data_from_csv(input_file):
    """Load latitude and longitude data from CSV file."""
    latitudes = []
    longitudes = []
    with open(input_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            latitudes.append(float(row['Latitude']))
            longitudes.append(float(row['Longitude']))
    return np.array(latitudes), np.array(longitudes)

def filter_results(input_file, output_file):
    # Bounding box coordinates (in degrees)
    lat1, lon1 = 40.66673, 20.58196
    lat2, lon2 = 50.74973, -12.64459
    lat3, lon3 = -10.09096, 50.71009
    lat4, lon4 = -20.32309, 100.79778

    lat_min = min(lat1, lat2, lat3, lat4)
    lat_max = max(lat1, lat2, lat3, lat4)
    lon_min = min(lon1, lon2, lon3, lon4)
    lon_max = max(lon1, lon2, lon3, lon4)

    print(f"Bounding box: Lat ({lat_min}, {lat_max}), Lon ({lon_min}, {lon_max})")

    # Load data from CSV file
    latitudes, longitudes = load_data_from_csv(input_file)
    num_elements = len(latitudes)
    print(f"Data loaded: {num_elements} entries")

    # Transfer data to GPU
    latitudes_cp = cp.asarray(latitudes)
    longitudes_cp = cp.asarray(longitudes)
    mask_cp = cp.zeros(num_elements, dtype=cp.int32)

    # Define CUDA kernel grid and block size
    block_size = 256
    grid_size = (num_elements + block_size - 1) // block_size

    # Launch CUDA kernel
    start_time = time.time()
    filter_kernel((grid_size,), (block_size,), (latitudes_cp, longitudes_cp,
                                                lat_min, lat_max,
                                                lon_min, lon_max,
                                                mask_cp, num_elements))
    cp.cuda.runtime.deviceSynchronize()  # Ensure the GPU has completed processing
    end_time = time.time()

    # Retrieve filtered data from GPU
    mask_np = cp.asnumpy(mask_cp)
    filtered_indices = np.where(mask_np == 1)[0]

    print(f"Filtering completed: {len(filtered_indices)} entries matched")

    # Write the filtered results to a new CSV file
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['Longitude', 'Latitude', 'Altitude']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        with open(input_file, 'r') as csvfile:
            reader = csv.DictReader(csvfile)
            for i, row in enumerate(reader):
                if i in filtered_indices:
                    writer.writerow({
                        'Longitude': row['Longitude'],
                        'Latitude': row['Latitude'],
                        'Altitude': row['Altitude']
                    })

    print(f"Filtering execution time: {end_time - start_time:.2f} seconds")

# Input and output files
input_file = 'satellite_latlongalt_gpu.csv'
output_file = 'filtered_satellite_positions_gpu.csv'

# Call the function to filter results based on user input
filter_results(input_file, output_file)
